In [1]:
import fs from 'node:fs'
const source = fs.readFileSync('../example.signup','utf-8')
source

# max 10 persons, and 202203 must has 3 persons.
must current < 10
must new in 2022
just new in 202203
a = current in 202203
b = a + 10 - current
must b >= 3
# ok!


In [2]:
const lines = source.split(/[\r\n]+/)
lines

[
  '# max 10 persons, and 202203 must has 3 persons.',
  'must current < 10',
  'must new in 2022',
  'just new in 202203',
  'a = current in 202203',
  'b = a + 10 - current',
  'must b >= 3',
  '# ok!'
]


In [3]:
function update(acc, neo){
    acc[acc.length-1]=neo;
}
function process(acc, char){
    if(char==='&'||char==='|'){
        if(acc.at(-1)===char){
            update(acc, char+char)
        }else{
            acc.push(char)
        }
    }else if(char===' '||char==='\t'){
        if(acc.at(-1)!==' '){
            acc.push(' ')
        }
    }else if(char==='='){
        const last = acc.at(-1);
        if(last==='>'||last==='<'||last==='='||last==='!'){
            update(acc,last+char)
        }else{
            acc.push(char)
        }
    }else if(char==='+'||char==='-'||char==='*'||char==='/'||char==='#'||char==='!'||char==='>'||char==='<'){
        acc.push(char)
    }else{
        if(/[a-zA-Z0-9]/.test(acc.at(-1))){
           update(acc,acc.at(-1)+char)
        }else{
            acc.push(char)
        }
    }
}

function lexer(source: string){
    let acc=[' '];
    for(const c of source){
        process(acc,c)
    }
    return acc.filter(v=>v!==' ').map(v=>Number.isFinite(+v)?+v:v).map(v=>v==='in'?'&':v);
}

lexer(' must a >= 0 &&b<0+1')

[
  'must', 'a',  '>=',
  0,      '&&', 'b',
  '<',    0,    '+',
  1
]


In [6]:
import type * as ast from '../ast.d.ts'

class Parser{

ctx={}
    
wantInt(from){
    if(from.valueType==='int') return from;
    if(from.valueType==='union') return {
        type:'union-to-int',
        from,
        valueType:'int'
    }
    throw new Error(`Cannot convert ${from.valueType} to int`)
}

wantBoolean(from){
    if(from.valueType==='boolean') return from;
    if(from.valueType==='int'||from.valueType==='union') return{
        type:'int-to-boolean',
        from:this.wantInt(from),
        valueType:'boolean'
    }
    throw new Error(`Cannot convert ${from.valueType} to boolean`)
}

parse(line){
    line = line.trim()
    if(line.length===0){
        return;
    }
    if(line[0]==='#'){
        return {
            type:'comment',
            value:line.slice(1)
        }
    }
    const tokens=lexer(line);
    if(typeof tokens[0]==='string'&&['must','just','return'].includes(tokens[0])){
        return {
            type: tokens[0],
            value: this.wantBoolean(this.parse1(tokens.slice(1))),
            valueType:'boolean'
        }
    }
    if(tokens[1]==='='){
       const v= {
           type:'assignment',
           id:tokens[0],
           value:this.parse1(tokens.slice(2))
       }
       this.ctx[v.id]=v.value.valueType
        return v
    }
    throw new Error('unknown statement')
}  
    
parse1(tokens){
    const i = tokens.findLastIndex(v=>['&&','||'].includes(v));
    if(i===-1){
        return this.parse2(tokens);
    }else{
        return {
            type:tokens[i],
            left:this.wantBoolean(this.parse1(tokens.slice(0,i))),
            right:this.wantBoolean(this.parse2(tokens.slice(i+1))),
            valueType:'boolean'
        }
    }
}

parse2(tokens){
    if(tokens[0]==='!'){
        return {
            type:'!',
            value:this.wantBoolean(this.parse3(tokens.slice(1))),
            valueType:'boolean'
        }
    }
    return this.parse3(tokens)
}
parse3(tokens){
    const i = tokens.findIndex(v=>['>','<','==','>=','<='].includes(v));
    if(i===-1){
        return this.parse4(tokens)
    }else{
        let left = this.wantInt(this.parse4(tokens.slice(0,i)))
        const right=this.wantInt(this.parse4(tokens.slice(i+1)))
        //…todo
        return {
            type: tokens[i],
            left,
            right,
            valueType:'boolean'
        }
    } 
}

commonRParser(next,list){
const func=function (tokens){
    const i = tokens.findLastIndex(v=>list.includes(v))
    if(i===-1){
        return next(tokens)
    }
    const left = func(tokens.slice(0,i))
    const right=func(tokens.slice(i+1))
    return {
        type: tokens[i],
        left,
        right
    }
}
return func
}
parse6=this.commonRParser(this.parse7.bind(this),['&','|'])

parse5=this.commonRParser(this.parse6.bind(this),['*','/'])

parse4=this.commonRParser(this.parse5.bind(this),['+','-'])

parse7(tokens){
    if(tokens.length!==1){
        throw new Error('error when parsing '+tokens.join(""))
    }
    const token = tokens[0]
    if(typeof token==='number'){
        if(token<2000){
        return {
            type:'numeric-literal',
            value:token,
            valueType:'int'
        }
        }else{
        return {
            type: 'union-literal',
            value:token,
            valueType:'union'
        }
        }
    }
    return {
        type: 'id',
        name: token,
        valueType: this.ctx[token]??'undefined'
    }
}
}

const p =new Parser()
p.parse('a=14')
const r = p.parse('must a>2020320')

fs.writeFileSync("../dist/result.json", JSON.stringify(r), "utf-8")

r

{
  type: 'must',
  value: {
    type: '>',
    left: { type: 'id', name: 'a', valueType: 'int' },
    right: { type: 'union-to-int', from: [Object], valueType: 'int' },
    valueType: 'boolean'
  },
  valueType: 'boolean'
}


In [5]:
+'114'

114
